### Define Documents

In [6]:
df = pd.read_csv('../../pico_vectors/preprocess/pico_nonoverlapping_tokenized.csv')
docs = df.abstract.ix[:1000].tolist()

for doc in docs[:5]:
    print '*'*80
    print
    print doc
    print

********************************************************************************

in recent years the treatment of primary nocturnal enuresis ( pne ) with desmopressin ( ddavp ) has been promising . the route of administration until now had been intranasal , but because the tablets were introduced for the treatment of diabetes insipidus they have also become available for the treatment of pne . to find the optimal dosage of desmopressin tablets and to compare desmopressin 's efficacy with placebo in a group of adolescents with severe monosymptomatic enuresis . the long-term safety of desmopressin was also studied in the same group of patients . the effect of oral desmopressin ( qqq ) ( ddavp tablets , minirin ) was investigated in qqq adolescents ( ages qqq to qqq years ) with severe monosymptomatic nocturnal enuresis . the first part of the dose-ranging study comprised a single-blind dose titration period , followed by a double-blind , crossover efficacy period comparing desmopressin 

### Define Generator That Yields Documents

In [7]:
from gensim.models.doc2vec import LabeledSentence

class DocumentIterator:
    """Class for yielding documents for use with doc2vec
    
    See usage below.
    
    """
    def __init__(self, texts):
        """Prepares `texts` for use with doc2vec
        
        Parameters
        ----------
        texts : list of strings where each string is a document
        
        Assumes `texts` has already been tokenized so calling `split()` on
        each text produces correct tokenization.
        
        """
        self.texts = [text.split() for text in texts]
        
    def __iter__(self):
        for i, text in enumerate(self.texts):
            yield LabeledSentence(words=text, tags=[i])

### Define `doc2vec` Model

In [8]:
%%time

from gensim.models import Doc2Vec

doc_iterator = DocumentIterator(docs)
model = Doc2Vec(doc_iterator)

model

CPU times: user 4.14 s, sys: 37.7 ms, total: 4.18 s
Wall time: 4.15 s


### Estimate Vectors

In [9]:
%%time

model.train(doc_iterator)

CPU times: user 4.15 s, sys: 25.2 ms, total: 4.17 s
Wall time: 4.32 s


1196020

### Access Learned Document Vectors

In [10]:
model.docvecs[0].shape

(300,)

### Infer Held-Out Document Vectors

In [11]:
unseen_docs = df.abstract.ix[1000:1005].tolist()

for doc in unseen_docs[:5]:
    print '*'*80
    print
    print doc
    print

********************************************************************************

a randomized , single-center , double-blind , parallel-group , placebo-controlled trial evaluated the whitening efficacy and safety of qqq , twice-daily use of a qqq % hydrogen peroxide tooth-bleaching gel delivered on polyethylene film . efficacy was based on change in vita shade scores from baseline to the end of treatment . thirty-three patients in each group completed treatment . use of the peroxide-containing gel led to a mean change in baseline vita shade score of qqq +/- qqq , compared with a change of qqq +/- qqq after use of a placebo gel . after adjustment for baseline scores , the mean difference in shade change between the peroxide gel-treated group and placebo-treated group was qqq +/- qqq ( p < qqq ) . both treatments were generally well tolerated . the strips offer ease of use , comfort , and shorter duration of wear compared with other at-home bleaching systems .

*************************

In [13]:
inferred_vecs = np.array([model.infer_vector(doc) for doc in unseen_docs])

inferred_vecs

array([[ 0.07745719, -0.15868787,  0.14197631, ...,  0.33927324,
         0.69629043, -0.69323844],
       [ 0.3235735 , -0.37674075, -0.0168117 , ...,  0.50764775,
         1.17010558, -1.07063079],
       [ 0.22329953, -0.18562463,  0.11352905, ...,  0.48121351,
         0.92684823, -0.80472386],
       [ 0.22682899, -0.21429279,  0.11964899, ...,  0.48293811,
         0.88048631, -0.76078665],
       [ 0.19689141, -0.24297367,  0.11889167, ...,  0.52631259,
         0.82017297, -0.71348065],
       [ 0.26055565, -0.34508079,  0.07482557, ...,  0.4912422 ,
         1.09229517, -0.96335256]], dtype=float32)

In [14]:
inferred_vecs.shape

(6, 300)